<a href="https://colab.research.google.com/github/DYuusha/Cjs/blob/master/plant_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#------ Imports --------
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras import layers
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)


In [ ]:
_URL = 'https://firebasestorage.googleapis.com/v0/b/agrocare-thesis.appspot.com/o/Plant_leave_diseases_dataset_without_augmentation.zip?alt=media&token=a1af7ba6-932a-43f4-9ddd-ab4dda76d2c7'

zip_dir = tf.keras.utils.get_file('Plant_leave_diseases_dataset_without_augmentation.zip', origin=_URL, extract=True)

In [ ]:
base_dir = os.path.join(os.path.dirname(zip_dir), 'Plant_leave_diseases_dataset_without_augmentation')

In [ ]:
train_dir = os.path.join(base_dir, 'training')
validation_dir = os.path.join(base_dir, 'validation')

In [ ]:
cd /root/.keras/datasets/Plant_leave_diseases_dataset_without_augmentation/training

/root/.keras/datasets/Plant_leave_diseases_dataset_without_augmentation/training


In [ ]:
classes = [
'Tomato___Bacterial_spot',
'Tomato___Early_blight',
'Tomato___healthy',
'Tomato___Late_blight',
'Tomato___Leaf_Mold',
'Tomato___Septoria_leaf_spot',
'Tomato___Spider_mites Two-spotted_spider_mite',
'Tomato___Target_Spot',
'Tomato___Tomato_mosaic_virus',
'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
]

import glob
import shutil
import matplotlib.pyplot as plt

for cl in classes:
  img_path = os.path.join(train_dir, cl)
  images = glob.glob(img_path + '/*.JPG')
  print("{}: {} Images".format(cl, len(images)))
  num_train = int(round(len(images)*0.8))
  train, val = images[:num_train], images[num_train:]

  for t in train:
    if not os.path.exists(os.path.join(base_dir, 'train', cl)):
      os.makedirs(os.path.join(base_dir, 'train', cl))
    shutil.move(t, os.path.join(base_dir, 'train', cl))
  
  for v in val:
    if not os.path.exists(os.path.join(base_dir, 'val', cl)):
      os.makedirs(os.path.join(base_dir, 'val', cl))
    shutil.move(v, os.path.join(base_dir, 'val', cl))

Tomato___Bacterial_spot: 1599 Images


Error: ignored

In [ ]:
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

print(train_dir)
print(val_dir)

/root/.keras/datasets/Plant_leave_diseases_dataset_without_augmentation/train
/root/.keras/datasets/Plant_leave_diseases_dataset_without_augmentation/val


In [ ]:
IMAGE_RES = 256

def format_image(image, label):
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return image, label

BATCH_SIZE = 32

In [ ]:
image_gen_train = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=45,
                    width_shift_range=.15,
                    height_shift_range=.15,
                    horizontal_flip=True,
                    zoom_range=0.5
                    )


training_set = image_gen_train.flow_from_directory(
                                                batch_size=BATCH_SIZE,
                                                directory=train_dir,
                                                shuffle=True,
                                                target_size=(IMAGE_RES,IMAGE_RES),
                                                class_mode='sparse'
                                                )

Found 8497 images belonging to 10 classes.


In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

In [ ]:
image_gen_val = ImageDataGenerator(rescale=1./255)

validation_set = image_gen_val.flow_from_directory(batch_size=BATCH_SIZE,
                                                 directory=val_dir,
                                                 target_size=(IMAGE_RES, IMAGE_RES),
                                                 class_mode='sparse')
plt.show()

Found 2126 images belonging to 10 classes.


In [ ]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,
                                   input_shape=(IMAGE_RES, IMAGE_RES, 3))

In [ ]:
#Freezing transfer learning model to prevent training and overfitting
feature_extractor.trainable = False

In [ ]:
num_classes = 0 
for i in classes:
  num_classes+=1

print('NUM CLASSES',num_classes)

NUM CLASSES 10


In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3,3),  padding="same", activation='relu', input_shape=(IMAGE_RES,IMAGE_RES,3)))
model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Conv2D(64, (3,3),  padding="same", activation='relu'))
model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Conv2D(128, (3,3),  padding="same", activation='relu'))
model.add(tf.keras.layers.Conv2D(128, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (4, 4)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(num_classes))

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_46 (Conv2D)           (None, 256, 256, 32)      896       
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 254, 254, 32)      9248      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 127, 127, 32)      0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 127, 127, 64)      18496     
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 125, 125, 64)      36928     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 62, 62, 64)       

In [ ]:
#Training model 
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

EPOCHS = 30

history = model.fit(training_set,
                    epochs=EPOCHS,
                    validation_data=validation_set)

Epoch 1/30
266/266 [==============================] - 125s 471ms/step - loss: 1.9018 - accuracy: 0.3068 - val_loss: 1.3589 - val_accuracy: 0.4962
Epoch 2/30
266/266 [==============================] - 125s 469ms/step - loss: 1.3683 - accuracy: 0.5091 - val_loss: 1.0585 - val_accuracy: 0.6406
Epoch 3/30
266/266 [==============================] - 124s 467ms/step - loss: 1.1362 - accuracy: 0.5972 - val_loss: 0.9458 - val_accuracy: 0.6679
Epoch 4/30
266/266 [==============================] - 124s 468ms/step - loss: 0.8747 - accuracy: 0.6953 - val_loss: 1.3278 - val_accuracy: 0.6538
Epoch 5/30
266/266 [==============================] - 125s 468ms/step - loss: 0.7466 - accuracy: 0.7406 - val_loss: 0.7258 - val_accuracy: 0.7700
Epoch 6/30
266/266 [==============================] - 124s 467ms/step - loss: 0.6248 - accuracy: 0.7886 - val_loss: 0.4871 - val_accuracy: 0.8401
Epoch 7/30
266/266 [==============================] - 124s 468ms/step - loss: 0.5723 - accuracy: 0.8055 - val_loss: 0.5610 -

In [ ]:
model.save_weights("model_weights.h5")

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
export_path_keras = './1592281759.h5'

reloaded = tf.keras.models.load_model(
  export_path_keras, 
  # `custom_objects` tells keras how to load a `hub.KerasLayer`
  custom_objects={'KerasLayer': hub.KerasLayer})

reloaded.summary()

In [ ]:
import time

t = time.time()

export_path_sm = "./{}".format(int(t))
print(export_path_sm)

tf.saved_model.save(reloaded, export_path_sm)

In [ ]:

converter = tf.lite.TFLiteConverter.from_saved_model(export_path_sm)
converter.optimizations = [tf.lite.Optimize.DEFAULT]


converter.inference_input_type = tf.uint8  # or tf.uint8
converter.inference_output_type = tf.uint8

tflite_quant_model = converter.convert()

open("converted_model.tflite", "wb").write(tflite_quant_model)

In [ ]:
class_names = np.array(classes)

print(class_names)

In [ ]:
image_batch, label_batch = next(iter(validation_set))

predicted_batch = model.predict(image_batch)
predicted_batch = tf.squeeze(predicted_batch).numpy()
predicted_ids = np.argmax(predicted_batch, axis=-1)
predicted_class_names = class_names[predicted_ids]

print(predicted_class_names)

In [ ]:
print("Labels:           ", label_batch)
print("Predicted labels: ", predicted_ids)

In [ ]:
plt.figure(figsize=(10,9))
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.subplots_adjust(hspace = 0.3)
  plt.imshow(image_batch[n])
  color = "blue" if predicted_ids[n] == label_batch[n] else "red"
  plt.title(predicted_class_names[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (blue: correct, red: incorrect)")